<a href="https://colab.research.google.com/github/jansoe/ANNet/blob/main/SentenceTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Few Shot Text Classification RAFT: Sentence Transformer

### Benötigte Bibliotheken installieren/laden

In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
!pip install sentence_transformers --q
import sentence_transformers as st

     |████████████████████████████████| 85 kB 3.2 MB/s 
     |████████████████████████████████| 5.3 MB 59.7 MB/s 
     |████████████████████████████████| 1.3 MB 50.6 MB/s 
     |████████████████████████████████| 163 kB 66.5 MB/s 
     |████████████████████████████████| 7.6 MB 52.9 MB/s 


In [ ]:
!pip install datasets --q
import datasets

     |████████████████████████████████| 441 kB 12.1 MB/s 
     |████████████████████████████████| 212 kB 65.7 MB/s 
     |████████████████████████████████| 115 kB 70.1 MB/s 
     |████████████████████████████████| 127 kB 73.4 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

## Daten laden

In [ ]:
training_data = datasets.load_dataset('ought/raft', name='twitter_complaints')

train_data = training_data['train'].to_pandas()
train_data.Label = 2 - train_data.Label

Extracting data files:   0%|          | 0/11 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3399 [00:00<?, ? examples/s]

Dataset raft downloaded and prepared to /root/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## Modell laden

#### Entweder: Pretrained Language Model

In [ ]:
sentence_encoder = st.SentenceTransformer('microsoft/mpnet-base')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/microsoft_mpnet-base were not used when initializing MPNetModel: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing MPNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/microsoft_mpnet-base and are newly initialized: ['mpnet.pooler.dense.weight', 'mpnet.pooler.dense.bias']
You sho

#### Oder: Pretrained Sentence Similarity Model

In [ ]:
sentence_encoder2 = st.SentenceTransformer('paraphrase-mpnet-base-v2')
# Trained on the following datasets:
#      AllNLI, sentence-compression, SimpleWiki, altlex, msmarco-triplets, quora_duplicates,
#      coco_captions,flickr30k_captions, yahoo_answers_title_question, S2ORC_citation_pairs,
#      stackexchange_duplicate_questions, wiki-atomic-edits

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/594 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

#### Tweets encodieren

In [ ]:
encoded_sentence_train = sentence_encoder.encode(train_data['Tweet text'].values)

In [ ]:
encoded_sentence_train.shape

(50, 768)

#### Logistic Regression for Tweet Classification

In [ ]:
model =  LogisticRegression()
model.fit(
    encoded_sentence_train,
    train_data.Label
)

LogisticRegression()

### Modell auf Testdaten testen

In [ ]:
fulldata = pd.read_csv(
    'https://raw.githubusercontent.com/danielpreotiuc/complaints-social-media/master/complaints-data.csv',
    names=['id', 'Tweet text', 'Label', 'category']
)

test_data = fulldata[~fulldata['Tweet text'].isin(training_data['train']['Tweet text'])]

Tweets encodieren ...

In [ ]:
encoded_sentence_test = sentence_encoder.encode(test_data['Tweet text'].values)

... und klassifizieren

In [ ]:
p_test = model.predict(encoded_sentence_test).squeeze()

#### Resultat

In [ ]:
accuracy = ((p_test >0.5) == test_data.Label).mean()
px.bar(
    y= p_test-0.5,
    color=test_data.Label,
    template='plotly_dark',
    text = test_data["Tweet text"],
    title = f'Accuracy {accuracy:.3f}',
    labels = {'color': 'Label', 'y': 'p-0.5'}
)